In [2]:
import pandas as pd

import numpy as np

import pickle

# Load Datasets

### Load Biological Process Terms

In [3]:
trainBP_terms=pd.read_pickle('../../../data/DeepGo/train-bp.pkl')

trainBP_terms=trainBP_terms[trainBP_terms['orgs']=='9606']

testBP_terms=pd.read_pickle('../../../data/DeepGo/test-bp.pkl')

testBP_terms=testBP_terms[testBP_terms['orgs']=='9606']

trainBP_terms.columns=['uniprot_ac', 'gos', 'labels', 'ngrams', 'proteins', 'sequences','orgs', 'embeddings']

testBP_terms.columns=['uniprot_ac', 'gos', 'labels', 'ngrams', 'proteins', 'sequences','orgs', 'embeddings']

# map uniprot to String

* STring IDs are required for embedding generation

In [4]:
import sys
sys.path.append('../../../utils/')

In [6]:
from  utils import UniprotID_to_StringId

### Convert Train IDs

In [7]:
dfTrain=UniprotID_to_StringId(trainBP_terms)

Not Matched:  286


/home/ubuntuadmin/anaconda37/lib/python3.7/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


In [8]:
dfTrain.head()

,uniprot_ac,gos,labels,ngrams,proteins,sequences,orgs,embeddings,uniprot_ac_uniprot_id,string_id
0,P31946,"[GO:0019904, GO:0019899, GO:0043488, GO:000582...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4331, 6603, 4049, 976, 3504, 6070, 1398, 3954...",1433B_HUMAN,MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLL...,9606,"[-0.690246, -1.16305, 0.082013, -1.17365, 0.89...",P31946|1433B_HUMAN,9606.ENSP00000361930
1,P62258,"[GO:1901016, GO:0086013, GO:0021762, GO:004698...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ...","[4043, 855, 1084, 5663, 1250, 998, 3960, 7194,...",1433E_HUMAN,MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLS...,9606,"[-0.707461, -0.440626, 0.28608, -1.23422, 0.47...",P62258|1433E_HUMAN,9606.ENSP00000264335
2,Q04917,"[GO:0042921, GO:0006713, GO:0019899, GO:001708...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4103, 2055, 1084, 5674, 1470, 5390, 3794, 387...",1433F_HUMAN,MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLS...,9606,"[-0.595342, -0.547682, 0.575142, -0.983365, 1....",Q04917|1433F_HUMAN,9606.ENSP00000248975
3,P61981,"[GO:0000086, GO:0045664, GO:0016020, GO:004816...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4343, 6855, 1084, 5674, 1470, 5398, 3954, 706...",1433G_HUMAN,MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLS...,9606,"[-0.661601, 0.378197, 0.508445, -0.970525, 0.4...",P61981|1433G_HUMAN,9606.ENSP00000306330
4,P31947,"[GO:0001836, GO:0008630, GO:0005829, GO:000716...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4075, 1481, 5616, 310, 6188, 3754, 3069, 5361...",1433S_HUMAN,MERASLIQKAKLAEQAERYEDMAAFMKGAVEKGEELSCEERNLLSV...,9606,"[-0.305037, -0.397102, 0.144294, -0.528591, 0....",P31947|1433S_HUMAN,9606.ENSP00000340989


### Convert Test IDs

In [9]:
dfTest=UniprotID_to_StringId(testBP_terms)

Not Matched:  46


# Load TripletProt Network and generate embeddings

In [10]:
from  utils import generate_tripletProt_embeddings

### Generate Train Embeddings

In [11]:
trainProtein_weights=generate_tripletProt_embeddings(dfTrain)







number of found:  7486


In [12]:
trainProtein_weights.shape

(7772, 64)

#### Generate Test Embeddings

In [13]:
testProtein_weights=generate_tripletProt_embeddings(dfTest)

number of found:  1921


In [14]:
testProtein_weights.shape

(1967, 64)

#### Prepare Input Data

In [15]:
X_train=trainProtein_weights

X_train=np.array(X_train)

In [16]:
Ytrain=dfTrain[['labels']]

Ytrain=Ytrain.labels.apply(pd.Series)

Ytrain=np.array(Ytrain)

Ytrain.shape

(7772, 932)

In [17]:
X_test=testProtein_weights

X_test=np.array(X_test)

In [18]:
Ytest=dfTest[['labels']]

Ytest=Ytest.labels.apply(pd.Series)

Ytest=np.array(Ytest)

Ytest.shape

(1967, 932)

In [19]:
num_classes=Ytrain.shape[1]

### Train and Evaluate the model

In [21]:
from sklearn.model_selection import StratifiedKFold

from  model import feedForward_classifier

import keras

from sklearn.metrics import label_ranking_average_precision_score

from numpy import arange

from sklearn.metrics import f1_score


In [22]:
random_seed=13

In [25]:
model=None
embedding_size=64
model = feedForward_classifier(num_classes,embedding_size)

In [26]:
history=model.fit(X_train, Ytrain, epochs=100, batch_size=32,verbose=1)
YtestPredicted_raw=model.predict(X_test)
avePrec =label_ranking_average_precision_score(Ytest, YtestPredicted_raw) 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
7772/7772 [==============================] - 1s 81us/step - loss: 0.2340
Epoch 2/100
7772/7772 [==============================] - 0s 44us/step - loss: 0.1326
Epoch 3/100
7772/7772 [==============================] - 0s 44us/step - loss: 0.1217
Epoch 4/100
7772/7772 [==============================] - 0s 44us/step - loss: 0.1165
Epoch 5/100
7772/7772 [==============================] - 0s 44us/step - loss: 0.1133
Epoch 6/100
7772/7772 [==============================] - 0s 44us/step - loss: 0.1112
Epoch 7/100
7772/7772 [==============================] - 0s 44us/step - loss: 0.1095
Epoch 8/100
7772/7772 [==============================] - 0s 44us/step - loss: 0.1083
Epoch 9/100
7772/7772 [==============================] - 0s 44us/step - loss: 0.1073
Epoch 10/100
7772/7772 [==============================] - 0s 44us/step - loss: 0.1064
Epoch 11/100
7772/7772 [==============================]

7772/7772 [==============================] - 0s 44us/step - loss: 0.0939
Epoch 93/100
7772/7772 [==============================] - 0s 44us/step - loss: 0.0938
Epoch 94/100
7772/7772 [==============================] - 0s 44us/step - loss: 0.0938
Epoch 95/100
7772/7772 [==============================] - 0s 44us/step - loss: 0.0937
Epoch 96/100
7772/7772 [==============================] - 0s 44us/step - loss: 0.0937
Epoch 97/100
7772/7772 [==============================] - 0s 44us/step - loss: 0.0936
Epoch 98/100
7772/7772 [==============================] - 0s 44us/step - loss: 0.0936
Epoch 99/100
7772/7772 [==============================] - 0s 44us/step - loss: 0.0936
Epoch 100/100
7772/7772 [==============================] - 0s 44us/step - loss: 0.0935


In [27]:
avePrec

0.4995345988369071

In [28]:
results={'treshold':[],'Average Precision':[],'F1 (micro)':[],'F1 (macro)':[],'Method':[]}

In [29]:
for treshold in arange(0.05,0.55,0.05):  
    YtestPredicted=None
    YtestPredicted=YtestPredicted_raw.copy()
    results['treshold'].append(treshold)
    results['Average Precision'].append(avePrec)
    YtestPredicted[YtestPredicted>=treshold]=1
    YtestPredicted[YtestPredicted<treshold]=0
    results['F1 (micro)'].append  (f1_score(Ytest, YtestPredicted, average='micro'))
    results['F1 (macro)'].append  (f1_score(Ytest, YtestPredicted, average='macro'))
    results['Method'].append ('TripletProt')


/home/ubuntuadmin/anaconda37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [30]:
df_results=pd.DataFrame(results)

In [31]:
df_results.head(50)

,treshold,Average Precision,F1 (micro),F1 (macro),Method
0,0.05,0.499535,0.317396,0.194042,TripletProt
1,0.10,0.499535,0.390645,0.224354,TripletProt
2,0.15,0.499535,0.429025,0.228960,TripletProt
3,0.20,0.499535,0.448706,0.220395,TripletProt
4,0.25,0.499535,0.457050,0.211765,TripletProt
5,0.30,0.499535,0.456187,0.199069,TripletProt
6,0.35,0.499535,0.449944,0.182735,TripletProt
7,0.40,0.499535,0.437714,0.168345,TripletProt
8,0.45,0.499535,0.421382,0.153327,TripletProt
9,0.50,0.499535,0.400206,0.136695,TripletProt


In [32]:
df_results.to_csv('../../../data/_Outputs/Final_TripletProt_BP.csv',index=False)